In [6]:
import os
os.environ['SPARK_HOME'] = '/opt/homebrew/opt/apache-spark/libexec/'
import findspark
findspark.init()

In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd
import requests
from io import StringIO

In [4]:
# Create a Spark session
spark = SparkSession.builder.appName('CA2').getOrCreate()

Picked up JAVA_TOOL_OPTIONS: -Dlog4j2.formatMsgNoLookups=true
Picked up JAVA_TOOL_OPTIONS: -Dlog4j2.formatMsgNoLookups=true
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/21 21:56:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
# Source: https://ec.europa.eu/eurostat/web/products-datasets/-/teiis500
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/teiis500?format=TSV"
res = requests.get(url)
res

<Response [200]>

In [21]:
eu_prod_construction_df = spark.createDataFrame(pd.read_csv(StringIO(res.text), delimiter='\t'))
eu_prod_construction_df.show(4)

+------------------------------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|freq,indic_bt,nace_r2,unit,geo\TIME_PERIOD|2022-04 |2022-05 |2022-06 |2022-07 |2022-08 |2022-09 |2022-10 |2022-11 |2022-12 |2023-01 |2023-02 |2023-03 |
+------------------------------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|                      M,PROD,F,I2015_SC...|  134.1 |  133.5 |  130.4 |  128.7 |  130.1 |  129.1 |  130.0 |  132.3 |  128.8 |  133.3 |  146.2 | 130.3 p|
|                      M,PROD,F,I2015_SC...|   98.8 |   99.4 |   98.7 |  100.5 |   99.1 |   98.1 |   98.3 |   99.9 |   99.3 |   97.9 |  100.6 |   98.4 |
|                      M,PROD,F,I2015_SC...|   89.9 |   91.8 |   90.4 |   89.9 |   90.2 |   90.0 |   90.6 |   90.9 |   89.4 |   89.8 |   89.5 |  90.2 p|
|                      M,PROD,F,I2015_SC...|  112.7 |  109.9 |  109.4 |  105.9 |  

In [22]:
# Shape of the DF
eu_prod_construction_df.count(), len(eu_prod_construction_df.columns)

(75, 13)

In [25]:
eu_prod_construction_df.describe().show()

23/05/21 22:14:40 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------------------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+
|summary|freq,indic_bt,nace_r2,unit,geo\TIME_PERIOD|         2022-04 |          2022-05 |          2022-06 |         2022-07 |          2022-08 |         2022-09 |         2022-10 |          2022-11 |          2022-12 |         2023-01 |          2023-02 |         2023-03 |
+-------+------------------------------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+
|  count|                                        75|               75|                75|                75|               75|                75|               75|            

In [26]:
eu_prod_construction_df.summary().show()

+-------+------------------------------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+
|summary|freq,indic_bt,nace_r2,unit,geo\TIME_PERIOD|         2022-04 |          2022-05 |          2022-06 |         2022-07 |          2022-08 |         2022-09 |         2022-10 |          2022-11 |          2022-12 |         2023-01 |          2023-02 |         2023-03 |
+-------+------------------------------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+
|  count|                                        75|               75|                75|                75|               75|                75|               75|            

In [31]:
# Taking out country, and unit of measurement to separate columns
eu_prod_construction_df.withColumn('country', col('freq,indic_bt,nace_r2,unit,geo\TIME_PERIOD')).apply(lambda x: x.split(',')[4])
eu_prod_construction_df.withColumn('stats', col('freq,indic_bt,nace_r2,unit,geo\TIME_PERIOD')).apply(lambda x: x.split(',')[3])

AttributeError: 'DataFrame' object has no attribute 'apply'